In [1]:
# bert
'''
原始语句：
    'Hello, how are you? I am Romeo.\n'
    'Hello, Romeo My name is Juliet. Nice to meet you.\n'
    'Nice meet you too. How are you today?\n'
    'Great. My baseball team won the competition.\n'
    'Oh Congratulations, Juliet\n'
    'Thanks you Romeo'
  ===========>
    [5, 13, 25, 20, 22, 6, 11], 
    [5, 11, 24, 21, 9, 15, 18, 16, 12, 20], 
    [18, 12, 20, 19, 13, 25, 20, 14], 
    [8, 24, 4, 17, 27, 23, 28], 
    [26, 10, 15], 
    [7, 20, 11]
============>(batch 操作）：每次随机抽取两个句子，一开始大概率是NotNext:False ,直到抽取到一半是IsNext时结束   
            比如抽到4,0
                  token_embeddign:           [1,       index_of_a,          2,       index_of_b ,                    2]
                  token_embeddign:           [1,       26, 10, 15,          2,       5, 13, 25, 20, 22, 6, 11,       2]
                  随机masked百分之15,随机抽取cand_maked_pos中的n_pred=2前两个位置作为mask language model替换       masked_pos=[5,10]    masked_tokens=[5,6]
                                             [1,       26, 10, 15,          2,       3, 13, 25, 20, 22, 3, 11,       2]
                                                                                     5--->3             6---->3
                  padding:
                    #input_ids     : [1, 26, 10, 15, 2, 3, 13, 25, 20, 22, 3, 11, 2,   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
                    #segment_ids   : [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
                    #masked_tokens : [5, 6, 0, 0, 0]
                    #masked_pos    : [5,10, 0, 0, 0]                                                               
                    
                            #input_ids:      token_embedding,   some token has been substituted
                            #segment_ids:    segment_embedding
                            # masked_tokens: the token which has been substituded                #作为target，Y
                            # masked_pos:    the token position which has been substituded
                            # isNext:        the sentence_a and sentence_b isNext or NotNext       #作为target，Y                                                                 
    return batch    #batch[i]= [input_ids, segment_ids, masked_tokens, masked_pos, True]
    
    embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)         (b,m,d)
    
    
    
'''

In [3]:

import math
import re
from random import *
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

In [4]:
text = (
    'Hello, how are you? I am Romeo.\n'
    'Hello, Romeo My name is Juliet. Nice to meet you.\n'
    'Nice meet you too. How are you today?\n'
    'Great. My baseball team won the competition.\n'
    'Oh Congratulations, Juliet\n'
    'Thanks you Romeo'
)


sentences = re.sub("[.,!?\\-]", '', text.lower()).split("\n")      # filter '.', ',', '?', '!'
# print(sentences)
'''
[ 'hello how are you i am romeo', 
'hello romeo my name is juliet nice to meet you', 
'nice meet you too how are you today', 
'great my baseball team won the competition', 
'oh congratulations juliet', 
'thanks you romeo'
]
'''
word_list = list(set(" ".join(sentences).split()))
# print(len(word_list))   #25
# print(word_list)
# ['too', 'to', 'romeo', 'congratulations', 'are', 'won', 'am', 'i', 'you', 'nice', 'how', 'my', 'great', 'name', 'meet', 'is', 'thanks', 'team', 'the', 'competition', 'oh', 'juliet', 'hello', 'today', 'baseball']

word2index = {'[PAD]' : 0, '[CLS]' : 1, '[SEP]' : 2, '[MASK]' : 3}
for i, w in enumerate(word_list):
    word2index[w] = i + 4
index2word = {i: w for i, w in enumerate(word2index)}
vocab_size = len(index2word)   #29


token_list = list()
for sentence in sentences:
    tmp=[word2index[i] for i in sentence.split()]
    # print(tmp)
    token_list.append(tmp)

print(token_list)
'''
[5, 13, 25, 20, 22, 6, 11], 
[5, 11, 24, 21, 9, 15, 18, 16, 12, 20], 
[18, 12, 20, 19, 13, 25, 20, 14], 
[8, 24, 4, 17, 27, 23, 28], 
[26, 10, 15], 
[7, 20, 11]
'''





[[23, 5, 20, 6, 26, 16, 21], [23, 21, 9, 18, 7, 13, 28, 24, 10, 6], [28, 10, 6, 27, 5, 20, 6, 14], [25, 9, 11, 22, 12, 15, 19], [8, 4, 13], [17, 6, 21]]


'\n[5, 13, 25, 20, 22, 6, 11], \n[5, 11, 24, 21, 9, 15, 18, 16, 12, 20], \n[18, 12, 20, 19, 13, 25, 20, 14], \n[8, 24, 4, 17, 27, 23, 28], \n[26, 10, 15], \n[7, 20, 11]\n'

In [10]:

# BERT Parameters

#    vocab_size = len(index2word) = 29      V
maxlen = 30          #两个句子cat之后的最大长度
batch_size = 6
max_pred = 5       # max tokens of prediction      最多的预测长度，取n_pred  = min(   5,len(input_ids)*0.15 )  # 15 % of tokens in one sentence
n_layers = 6
n_heads = 12
d_model = 768     #dimension  D
d_ff = 768 * 4  # 4*d_model, FeedForward dimension
d_k = d_v = 64  # dimension of K(=Q), V
n_segments = 2


# sample IsNext and NotNext to be same in small batch size
def make_batch():
    batch = []
    positive = negative = 0       #positive=num_of_isNext    negativa=num_of_NotNext

    tmpnum = 0
     #每次随机抽取两个句子，一开始大概率是NotNext:False ,直到抽取到一半是IsNext时结束          一共batch_size/2*2=batch_size=6次
    while positive != batch_size/2 or negative != batch_size/2:
    # if positive != batch_size / 2 or negative != batch_size / 2:
        tmpnum += 1
        print('index---------->', tmpnum)
        # print(len(sentences))        #6
        tokens_a_index, tokens_b_index = randrange(len(sentences)), randrange( len(sentences))  # sample random index in sentences
        # print(tokens_a_index, tokens_b_index)   # 4,0
        #tokens_a,tokens_b 为随机找的两个句子
        tokens_a, tokens_b = token_list[tokens_a_index], token_list[tokens_b_index]
        #                 [1,       index_of_a,          2,       index_of_b ,                    2]
        # <class 'list'>: [1,       26, 10, 15,          2,       5, 13, 25, 20, 22, 6, 11,       2]
        input_ids = [word2index['[CLS]']] + tokens_a + [word2index['[SEP]']] + tokens_b + [word2index['[SEP]']]
        #<class 'list'>:  [0,        0, 0, 0,            0,        1, 1, 1, 1, 1, 1, 1,            1 ]
        segment_ids =    [0] * (1 + len(tokens_a) + 1)        +    [1] * (len(tokens_b) + 1)



        # MASK LM                          round(13*0.15)=round(1.94)=2    #len(input_ids) 为两个句子合并之后的长度
        n_pred = min(max_pred, max(1, int(round(len(input_ids) * 0.15))))       # 15 % of tokens in one sentence     #2
        #<class 'list'>:  [          1, 2, 3,                     5, 6, 7, 8, 9, 10, 11]             去掉1,2之后剩下的token位置
        cand_maked_pos = [i for i, token in enumerate(input_ids)
                          if token != word2index['[CLS]'] and token != word2index['[SEP]']]
        shuffle(cand_maked_pos)        #<class 'list'>: [5, 10, 2, 9, 11, 3, 7, 1, 6, 8]

        masked_tokens, masked_pos = [], []
        for pos in cand_maked_pos[:n_pred]:       #随机抽取cand_maked_pos中的n_pred=2前两个位置作为mask language model替换   masked_tokens=[5,10]
            masked_pos.append(pos)                    # masked_pos=[5,10]
            masked_tokens.append(input_ids[pos])      #masked_tokens =[5,6]

            #<class 'list'>: [1,       26, 10, 15,          2,       5, 13, 25, 20, 22, 6, 11,       2]
            #<class 'list'>: [1,       26, 10, 15,          2,       3, 13, 25, 20, 22, 3, 11,       2]
            #         5----->3        6--------->3
            if random() < 0.8:  # 80%
                input_ids[pos] = word2index['[MASK]']     # make mask        word2index['[MASK]']=3
            elif random() < 0.5:  # 10%
                index = randint(0, vocab_size - 1)  # random index in vocabulary
                input_ids[pos] = word2index[index2word[index]]  # replace

        # Zero Paddings
        n_pad = maxlen - len(input_ids)        #30-13=17
        input_ids.extend([0] * n_pad)         #input_ids     : [1, 26, 10, 15, 2, 3, 13, 25, 20, 22, 3, 11, 2,   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        segment_ids.extend([0] * n_pad)       #segment_ids   : [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

        # Zero Padding (100% - 15%) tokens
        if max_pred > n_pred:       #5>2
            n_pad = max_pred - n_pred    #3
            masked_tokens.extend([0] * n_pad)  #masked_tokens : [5, 6, 0, 0, 0]
            masked_pos.extend([0] * n_pad)     #masked_pos    : [5,10, 0, 0, 0]


         #每次随机抽取两个句子，一开始大概率是NotNext:False ,直到抽取到一半是IsNext时结束
        if tokens_a_index + 1 == tokens_b_index and positive < batch_size / 2:
            print("True  IsNext",positive)
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True])  # IsNext
            positive += 1
        elif tokens_a_index + 1 != tokens_b_index and negative < batch_size / 2:
            print("False  NotNext",negative)
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False])  # NotNext
            negative += 1
    
    
    #input_ids:      token_embedding,   some token has been substituted
    #segment_ids:    segment_embedding
    # masked_tokens: the token which has been substituded                #作为target，Y
    # masked_pos:    the token position which has been substituded
    # isNext:        the sentence_a and sentence_b isNext or NotNext       #作为target，Y
     
    return batch         #batch[i]= [input_ids, segment_ids, masked_tokens, masked_pos, True]


# Proprecessing Finished

batch = make_batch()

for each in batch:
    print(each)

index----------> 1
False  NotNext 0
index----------> 2
False  NotNext 1
index----------> 3
False  NotNext 2
index----------> 4
index----------> 5
True  IsNext 0
index----------> 6
index----------> 7
index----------> 8
index----------> 9
index----------> 10
True  IsNext 1
index----------> 11
index----------> 12
True  IsNext 2
[[1, 23, 5, 20, 6, 26, 16, 21, 2, 17, 3, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [6, 21, 0, 0, 0], [10, 11, 0, 0, 0], False]
[[1, 17, 3, 21, 2, 23, 21, 9, 18, 7, 13, 28, 24, 3, 6, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [10, 6, 0, 0, 0], [13, 2, 0, 0, 0], False]
[[1, 3, 6, 21, 2, 17, 6, 21, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [17, 0, 0, 0, 

In [11]:
input_ids, segment_ids, masked_tokens, masked_pos, isNext = zip(*batch)
print(input_ids,"\n\n",segment_ids,"\n\n",masked_tokens,"\n\n",masked_pos,"\n",isNext)


input_ids, segment_ids, masked_tokens, masked_pos, isNext = \
    torch.LongTensor(input_ids),  torch.LongTensor(segment_ids), torch.LongTensor(masked_tokens), \
    torch.LongTensor(masked_pos), torch.LongTensor(isNext)


print(input_ids.size(),"\n\n",segment_ids.size(),"\n\n",masked_tokens.size(),"\n\n",masked_pos.size(),"\n",isNext.size(),isNext)


Epoch: 0001 cost = 35.173149


In [13]:
def get_attn_pad_mask(seq_q, seq_k):
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # batch_size x 1 x len_k(=len_q), one is masking
    return pad_attn_mask.expand(batch_size, len_q, len_k)  #(b,m,m) 



def gelu(x):
    "Implementation of the gelu activation function by Hugging Face"
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))





class Embedding(nn.Module):
    def __init__(self):
        super(Embedding, self).__init__()
        self.tok_embed = nn.Embedding(vocab_size, d_model)  # token embedding
        self.pos_embed = nn.Embedding(maxlen, d_model)  # position embedding
        self.seg_embed = nn.Embedding(n_segments, d_model)  # segment(token type) embedding
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x, seg):        #(input_ids, segment_ids)     (b,m),(b,m)
        seq_len = x.size(1)         #max_len=m=30
        pos = torch.arange(seq_len, dtype=torch.long)
        # print(pos)       tensor([  0,   1,   2,   3,   4,   5...........23, 24,  25,  26,  27,  28,  29])
        # (1,m) -------> (b,m)
        pos = pos.unsqueeze(0).expand_as(x) 
        embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
        # print(embedding)  #(b,m,d) =(6,30,768)
        # print(self.norm(embedding))     #(b,m,d) =(6,30,768)
        return self.norm(embedding)      #(b,m,d) 


class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k) # scores : [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is one.
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V)
        return context, attn




class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(d_model, d_k * n_heads)
        self.W_K = nn.Linear(d_model, d_k * n_heads)
        self.W_V = nn.Linear(d_model, d_v * n_heads)
    def forward(self, Q, K, V, attn_mask):
        # q: [batch_size x len_q x d_model], k: [batch_size x len_k x d_model], v: [batch_size x len_k x d_model]
        residual, batch_size = Q, Q.size(0)
        # (B, S, D) -proj-> (B, S, D) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        q_s = self.W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # q_s: [batch_size x n_heads x len_q x d_k]
        k_s = self.W_K(K).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # k_s: [batch_size x n_heads x len_k x d_k]
        v_s = self.W_V(V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # v_s: [batch_size x n_heads x len_k x d_v]

        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1) # attn_mask : [batch_size x n_heads x len_q x len_k]

        # context: [b,n_heads,m,d_v], attn: [b,n_heads,m,m]
        context, attn = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_heads * d_v) # context: [b,m, n_heads * d_v]
        output = nn.Linear(n_heads * d_v, d_model)(context)
        return nn.LayerNorm(d_model)(output + residual), attn      # output: [b,m,d]    attn: [b,n_heads,m,m]
 
 
 
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        # (b, m, d) -> (b, m, d_ff) -> (b, m, d) 
        return self.fc2(gelu(self.fc1(x)))



class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()
    def forward(self, enc_inputs, enc_self_attn_mask):
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask)   # enc_outputs:(b,m,d)     attn: [b,n_heads,m,m]
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs:(b,m,d) 
        return enc_outputs, attn      # enc_outputs:(b,m,d)     attn: [b,n_heads,m,m]



class BERT(nn.Module):
    def __init__(self):
        super(BERT, self).__init__()
        self.embedding = Embedding()     
        
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])
        
        self.fc = nn.Linear(d_model, d_model)
        self.activ1 = nn.Tanh()
        self.linear = nn.Linear(d_model, d_model)
        self.activ2 = gelu
        self.norm = nn.LayerNorm(d_model)
        self.classifier = nn.Linear(d_model, 2)
        # decoder is shared with embedding layer
        embed_weight = self.embedding.tok_embed.weight
        n_vocab, n_dim = embed_weight.size()
        self.decoder = nn.Linear(n_dim, n_vocab, bias=False)
        #decoder 和 token_embbedding 的parameter一样
        self.decoder.weight = embed_weight
        self.decoder_bias = nn.Parameter(torch.zeros(n_vocab))

    def forward(self, input_ids, segment_ids, masked_pos):
        output = self.embedding(input_ids, segment_ids)       #(b,m,d)
        enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids) #(b,m,m)
        
        for layer in self.layers:
            output, enc_self_attn = layer(output, enc_self_attn_mask)       # output:(b,m,d)     attn: [b,n_heads,m,m]
            
            
            
        #预测 isNext or NotNext
        # output : [b, m, d], attn : [b, n_heads, m,m]
        # it will be decided by first token(CLS)
        h_pooled = self.activ1(self.fc(output[:, 0])) # [b, d]
        logits_clsf = self.classifier(h_pooled) # [b, 2]


        #预测  marked_token   (b,m_m,1)------>(b,m_m,d)
        masked_pos = masked_pos[:, :, None].expand(-1, -1, output.size(-1)) # [b, m_m, d]   m_m:maxlen_marked
        # get masked position from final output of transformer.
        h_masked = torch.gather(output, 1, masked_pos) # masking position  [b, m_m, d] 
        h_masked = self.norm(self.activ2(self.linear(h_masked))) #[b, m_m, d] 
        logits_lm = self.decoder(h_masked) + self.decoder_bias # [b, m_m, v] 
        return logits_lm, logits_clsf                          # [b, m_m, v]  , [b, 2]
    
    
model = BERT()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


for epoch in range(1):
    optimizer.zero_grad()
    logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)
    loss_lm = criterion(logits_lm.transpose(1, 2), masked_tokens) # for masked LM
    loss_lm = (loss_lm.float()).mean()
    loss_clsf = criterion(logits_clsf, isNext) # for sentence classification
    loss = loss_lm + loss_clsf
    if (epoch ) % 10 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
    loss.backward()
    optimizer.step()


Epoch: 0001 cost = 35.173149


In [ ]:
'''
BERT(
  (embedding): Embedding(
    (tok_embed): Embedding(29, 768)
    (pos_embed): Embedding(30, 768)
    (seg_embed): Embedding(2, 768)
    (norm): LayerNorm(torch.Size([768]), eps=1e-05, elementwise_affine=True)
  )
  
  (layers): ModuleList(
    (0-5): EncoderLayer(           #6层encoderlayer
      (enc_self_attn): MultiHeadAttention(
        (W_Q): Linear(in_features=768, out_features=768, bias=True)
        (W_K): Linear(in_features=768, out_features=768, bias=True)
        (W_V): Linear(in_features=768, out_features=768, bias=True)
      )
      (pos_ffn): PoswiseFeedForwardNet(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
      )
    )
  )
  
  (fc): Linear(in_features=768, out_features=768, bias=True)
  (activ1): Tanh()
  (linear): Linear(in_features=768, out_features=768, bias=True)
  (norm): LayerNorm(torch.Size([768]), eps=1e-05, elementwise_affine=True)
  (classifier): Linear(in_features=768, out_features=2, bias=True)
  (decoder): Linear(in_features=768, out_features=29, bias=False)
)
'''


Hello, how are you? I am Romeo.
Hello, Romeo My name is Juliet. Nice to meet you.
Nice meet you too. How are you today?
Great. My baseball team won the competition.
Oh Congratulations, Juliet
Thanks you Romeo
['[CLS]', 'hello', 'how', 'are', 'you', 'i', 'am', 'romeo', '[SEP]', 'thanks', '[MASK]', '[MASK]', '[SEP]']
masked tokens list :  [6, 21]
predict masked tokens list :  []
isNext :  False
predict isNext :  False


In [16]:



# Predict mask tokens ans isNext
input_ids, segment_ids, masked_tokens, masked_pos, isNext = batch[0]
print(text)
print([index2word[w] for w in input_ids if index2word[w] != '[PAD]'])

logits_lm, logits_clsf = model(torch.LongTensor([input_ids]), \
                               torch.LongTensor([segment_ids]), torch.LongTensor([masked_pos]))
logits_lm = logits_lm.data.max(2)[1][0].data.numpy()
print('masked tokens list : ',[pos for pos in masked_tokens if pos != 0])
print('predict masked tokens list : ',[pos for pos in logits_lm if pos != 0])

logits_clsf = logits_clsf.data.max(1)[1].data.numpy()[0]
print('isNext : ', True if isNext else False)
print('predict isNext : ',True if logits_clsf else False)


Hello, how are you? I am Romeo.
Hello, Romeo My name is Juliet. Nice to meet you.
Nice meet you too. How are you today?
Great. My baseball team won the competition.
Oh Congratulations, Juliet
Thanks you Romeo
['[CLS]', 'hello', 'how', 'are', 'you', 'i', 'am', 'romeo', '[SEP]', 'thanks', '[MASK]', '[MASK]', '[SEP]']
masked tokens list :  [6, 21]
predict masked tokens list :  []
isNext :  False
predict isNext :  False


In [41]:
#(b,m) (b,m)   (b,m_m)  maxlen_maskedb